In [1]:
import os
import numpy as np
import tensorflow as tf
import datetime
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks

from sklearn.metrics import classification_report

tf.random.set_seed(0)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [2]:
X_train = np.load("../preprocess_data/train-pcap.npy")
y_train = np.load("../preprocess_data/train-labels.npy")
X_test = np.load("../preprocess_data/test-pcap.npy")
y_test = np.load("../preprocess_data/test-labels.npy")
X_train.shape, X_test.shape

((49986, 784), (5618, 784))

In [3]:
X_train = X_train.astype("float32").reshape(-1, 28, 28, 1)
X_test = X_test.astype("float32").reshape(-1, 28, 28, 1)

In [4]:
def get_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (5, 5), activation="relu", input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(64, (5, 5), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(1024))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(12))
    return model

In [5]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [6]:
model = get_model()
model.compile(loss=loss_func, optimizer=optimizer, metrics=["accuracy"])
start_time = datetime.datetime.now()
early_stopping =callbacks.EarlyStopping(monitor='val_accuracy', patience=20) 
model.fit(X_train, y_train, epochs=200, batch_size=128, verbose=1, validation_split=0.2, callbacks=[early_stopping])
print ('model_train_time：', datetime.datetime.now() - start_time)

Epoch 1/200
313/313 [==============================] - 2s 7ms/step - loss: 1.8604 - accuracy: 0.6493 - val_loss: 0.7791 - val_accuracy: 0.6827
Epoch 2/200
313/313 [==============================] - 2s 7ms/step - loss: 0.7984 - accuracy: 0.6651 - val_loss: 0.7536 - val_accuracy: 0.6786
Epoch 3/200
313/313 [==============================] - 2s 7ms/step - loss: 0.7541 - accuracy: 0.6788 - val_loss: 0.7128 - val_accuracy: 0.7135
Epoch 4/200
313/313 [==============================] - 2s 7ms/step - loss: 0.7268 - accuracy: 0.6911 - val_loss: 0.6789 - val_accuracy: 0.7259
Epoch 5/200
313/313 [==============================] - 2s 7ms/step - loss: 0.7032 - accuracy: 0.7120 - val_loss: 0.6876 - val_accuracy: 0.7181
Epoch 6/200
313/313 [==============================] - 2s 7ms/step - loss: 0.6709 - accuracy: 0.7285 - val_loss: 0.6467 - val_accuracy: 0.7381
Epoch 7/200
313/313 [==============================] - 2s 7ms/step - loss: 0.6409 - accuracy: 0.7433 - val_loss: 0.6252 - val_accuracy: 0.7429

Epoch 58/200
313/313 [==============================] - 2s 7ms/step - loss: 0.2845 - accuracy: 0.8699 - val_loss: 0.6950 - val_accuracy: 0.8298
Epoch 59/200
313/313 [==============================] - 2s 7ms/step - loss: 0.2881 - accuracy: 0.8673 - val_loss: 0.8635 - val_accuracy: 0.8178
Epoch 60/200
313/313 [==============================] - 2s 7ms/step - loss: 0.2797 - accuracy: 0.8702 - val_loss: 0.7086 - val_accuracy: 0.8221
Epoch 61/200
313/313 [==============================] - 2s 7ms/step - loss: 0.2934 - accuracy: 0.8657 - val_loss: 0.9061 - val_accuracy: 0.8410
Epoch 62/200
313/313 [==============================] - 2s 7ms/step - loss: 0.3113 - accuracy: 0.8624 - val_loss: 0.6958 - val_accuracy: 0.8404
Epoch 63/200
313/313 [==============================] - 2s 7ms/step - loss: 0.2942 - accuracy: 0.8689 - val_loss: 0.8961 - val_accuracy: 0.8206
Epoch 64/200
313/313 [==============================] - 2s 7ms/step - loss: 0.2959 - accuracy: 0.8713 - val_loss: 0.8012 - val_accuracy:

In [7]:
start_time = datetime.datetime.now()
y_pre = np.argmax(model.predict(X_test), axis=1)
print ('predict_time：', datetime.datetime.now() - start_time)
print(classification_report(y_test, y_pre, digits=4))

predict_time： 0:00:00.331225
              precision    recall  f1-score   support

           0     0.7000    0.4088    0.5161       137
           1     0.7317    0.4688    0.5714        64
           2     0.6780    0.8228    0.7434      1236
           3     0.7778    0.7179    0.7467        39
           4     0.7606    0.6067    0.6750       178
           5     0.8994    0.8672    0.8830      3720
           6     0.5000    0.2414    0.3256        29
           7     0.7500    0.6000    0.6667        15
           8     0.7714    0.5870    0.6667        46
           9     0.8636    0.7308    0.7917        26
          10     0.8200    0.7193    0.7664        57
          11     0.5657    0.7887    0.6588        71

    accuracy                         0.8231      5618
   macro avg     0.7348    0.6299    0.6676      5618
weighted avg     0.8299    0.8231    0.8226      5618

